In [ ]:
from tkinter import *
from PIL import Image, ImageDraw, ImageTk
import glob
from os import path

class PaintCanvas:
    def __init__(self,window):
        self.window = window
        self.old_x = None
        self.old_y = None
        self.penWidth = 13
        self.canvas_width = 400
        self.canvas_height = 150
        self.penColor = 'black'
        self.fg_color = (0,0,0) #for draw 
        self.bg_color = (255,255,255) #for draw
        self.createWidgets()

    def paint(self,e):
        if self.old_x and self.old_y:
            self.c.create_line(self.old_x,self.old_y,e.x,e.y,
                width=self.penWidth,fill=self.penColor,
                capstyle=ROUND,smooth=True
            )
            self.draw.line([self.old_x,self.old_y,e.x,e.y],self.fg_color, width=15)

        self.old_x = e.x
        self.old_y = e.y

    def reset(self,e):    #reseting or cleaning the canvas 
        self.old_x = None
        self.old_y = None 
    
    def createWidgets(self):
        white = (255,255,255)

        x,y = 280, 330
        ShowButton(self.window,"Pen",x,y,self.selectPen)
        ShowButton(self.window,"Eraser",x+60,y,self.selectEraser)
        ShowButton(self.window,"Clear",x+120,y,self.selectClear)
        ShowButton(self.window,"Save",x+180,y,self.saveImage)

        self.c = Canvas(self.window,width=self.canvas_width,height=self.canvas_height,bg='white')
        self.c.pack()
        self.c.place(x=100,y=365)

        self.c.bind('<B1-Motion>',self.paint)#drwaing the line 
        self.c.bind('<ButtonRelease-1>',self.reset)

        self.image1 = Image.new("RGB", (500, 150), self.bg_color)
        self.draw = ImageDraw.Draw(self.image1)
    
    def selectClear(self):
        self.c.delete("all")
        self.draw.line([0,0,500,500],self.bg_color, width=500)
    
    def selectPen(self):
        self.penWidth = 13
        self.penColor = "black"
    
    def selectEraser(self):
        self.penWidth = 15
        self.penColor = "white"
    
    def saveImage(self):
        Images_path = os.getcwd() + '\\Images\\'
        filename = Images_path+"answer.jpg"
        self.image1.save(filename)
        #answer = identifyImage()
        #print(answer)

def ShowText(window,Text,X,Y,Height, Width, FontSize, Font="Arial Bold"):
    canvas = tk.Canvas(root, height=360, width=1333)
    canvas.pack()
    
    bg_label = canvas.create_image((0,0), image=bg_img, anchor=tk.N+tk.W)
    label_cd = canvas.create_text((1030,120), text=str(tte), font="MSGothic 50 bold", fill="#652828")

    textLabel = Label(window, text=Text, font = (Font,FontSize), height=Height,width=Width)
    textLabel.pack()
    textLabel.place(x=X, y=Y)   

def ShowButton(window,text,X,Y,Command):
    panel = Button(window, text=text,command=Command)
    panel.pack()
    panel.place(x=X,y=Y)

def ShowImage(window,Path,X,Y,Height,Width):
    img = Image.open((Path))
    img = img.resize((Height, Width), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    panel = Label(window, image=img)
    panel.image = img
    panel.pack()
    panel.place(x=X,y=Y)

if __name__ == "__main__":
    window = Tk()
    window.geometry('+200+50') #show window starting from x-200 and y-50
    window.geometry('600x600') #set the geomentry of window
    window.title("Machine learning")
    # ShowImage(window,"background.jpg",0,0,600,600)
    # ShowText(window,"Write Your Answer",5,50,0,0,40)
    # ShowText(window,"11 + 12 =",50,150,0,0,60)
    Images_path = os.getcwd() + '\\Images\\'
    bg_img = Images_path + "background.jpg"
    
    img = Image.open((bg_img))
    img = img.resize((600, 600), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    canvas = Canvas(window, height=600, width=600)
    canvas.pack()
    bg_label = canvas.create_image((0,0), image=img, anchor=N+W)
    label1 = canvas.create_text((300,100), text="Write Your Answer", font="MSGothic 40 bold", fill="#652828")
    label2 = canvas.create_text((300,225), text="11 + 12 =", font="MSGothic 75 bold", fill="#652828")
    PaintCanvas(window)
    window.mainloop()

In [15]:
#impoting packages
import imageio
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2
from PIL import Image

# load the model
from keras.models import load_model
model = load_model("test_model.h5")

In [31]:
#image_segmentation
Images_path = os.getcwd() + '\\Images\\'

non_segmented_image_path = Images_path + 'answer.jpg'

img = cv2.imread(non_segmented_image_path)

#segmenting the image with counter from openCV
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 127, 255, 0)
contours, hierarchies = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print("Number of contours = " + str(len(contours)))
print("Format of hieraechy = [[Next, Previous, First_Child, Parent]]")

cv2.drawContours(img, contours, -1, (0, 255, 0), 1)
#cv2.imshow("img",img)

#deleting the images already in the directory
imageLocation = Images_path + 'Segmented_Images\\'
for file in os.listdir(imageLocation):
        os.remove(imageLocation+file) 

#sorted_ctrs = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0]  )

#removing unnecessary contours        
#hierarchy[0,i,3] #hierarchy=[[Next, Previous, First_Child, Parent]]
contours_list = []
for contour,hierarchy in zip(contours,hierarchies[0]):
    if hierarchy[3] == 0 or cv2.contourArea(contour) > 100 and hierarchy[2] != -1 :
        contours_list.append(contour)
        
#sorting contours
contours_list = sorted(contours_list, key=lambda ctr: cv2.boundingRect(ctr)[0]  )
print('Number of countours after filtering = ',len(contours_list[1:]) )        

#saving and naming Contours in images
for i,contour in enumerate(contours_list[1:]):
        (x, y, w, h) = cv2.boundingRect(contour)
        
        #padding images and cropping
        p = 10  
        x,y,w,h = x-p,y-p,w+p+10,h+p+10
        roi = img[y:y+h, x:x+w] #roi = region of image
        
        #saving the image
        name = imageLocation + 'roi'+str(i)+'.png'
        cv2.imwrite(name, roi)
        
        #saving image with aspect-ratio
        img2 = Image.open(name)
        w1,h1 = img2.size
        background = Image.open(Images_path + 'number_background.png')
        w2,h2 = background.size
        pasteLocation = int(w2/2) - int(w1/2), int(h2/2) - int(h1/2)
        background.paste(img2,pasteLocation)
        background.save(name,"PNG")

        #cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

Number of contours = 4
Format of hieraechy = [[Next, Previous, First_Child, Parent]]
Number of countours after filtering =  3


In [21]:
#image identification

predicted_number = []
for image in os.listdir(imageLocation):
    im = imageio.imread(imageLocation+image)
    #getting image
    #im = imageio.imread(imageLocation+'roi2.png')
    dim = (28,28)

    #preprocessing image
    im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
    #plt.imshow(im)
    gray = np.dot(im[...,:3], [0.299, 0.587, 0.114])
    gray = gray.reshape(1, 28, 28, 1)
    gray = 255-gray
    gray = gray/255

    # predict digit
    prediction = model.predict(gray)
    predicted_number.append(str(prediction.argmax()))
print('Predicted Number =', ''.join(predicted_number))

Predicted Number = 1
